In [154]:
import pandas as pd
import numpy as np
import torch
import time

In [231]:
# importing the dataset 
df_train = pd.read_excel('./title-classfication-train.xlsx')
df_test = pd.read_excel('./title-classfication-val.xlsx')

In [232]:
df_test = df_test.drop(columns=['Date'])
df_train = df_train.drop(columns=['Date'])


In [233]:
df_train["Trend"] = df_train["Trend"].replace("A","Positive")
df_train["Trend"] = df_train["Trend"].replace("B","Neutral")
df_train["Trend"] = df_train["Trend"].replace("C","Negative")

df_test["Trend"] = df_test["Trend"].replace("A","Positive")
df_test["Trend"] = df_test["Trend"].replace("B","Neutral")
df_test["Trend"] = df_test["Trend"].replace("C","Negative")

In [234]:
encoded_dict = {'Positive':0,'Neutral':1, 'Negative':2}
df_train['Trend'] = df_train.Trend.map(encoded_dict)
df_test['Trend'] = df_test.Trend.map(encoded_dict)

In [235]:
from tensorflow.keras.utils import to_categorical
import transformers
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [236]:
y_train = to_categorical(df_train.Trend)
y_test = to_categorical(df_test.Trend)

In [237]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [238]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.Title.tolist(),
    add_special_tokens=True,
    max_length=27,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.Title.tolist(),
    add_special_tokens=True,
    max_length=27,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [239]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [240]:
#########可能有問題

max_len = 27
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [241]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy')
# Compile the model
model.compile(optimizer = optimizer,loss = loss, metrics = metric)

In [242]:
train_history = model.fit(x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
  y = y_train,
  validation_data = ({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test),
  epochs=30,
  batch_size=36
)

Epoch 1/30
39/39 [==============================] - 161s 4s/step - loss: 1.0998 - balanced_accuracy: 0.4038 - val_loss: 0.9757 - val_balanced_accuracy: 0.5344
Epoch 2/30
39/39 [==============================] - 136s 4s/step - loss: 0.8229 - balanced_accuracy: 0.6296 - val_loss: 0.7726 - val_balanced_accuracy: 0.6461
Epoch 3/30
39/39 [==============================] - 136s 4s/step - loss: 0.5280 - balanced_accuracy: 0.7669 - val_loss: 0.6626 - val_balanced_accuracy: 0.7219
Epoch 4/30
39/39 [==============================] - 136s 4s/step - loss: 0.3091 - balanced_accuracy: 0.8794 - val_loss: 0.7044 - val_balanced_accuracy: 0.7328
Epoch 5/30
39/39 [==============================] - 137s 4s/step - loss: 0.1796 - balanced_accuracy: 0.9368 - val_loss: 0.7700 - val_balanced_accuracy: 0.7227
Epoch 6/30
39/39 [==============================] - 136s 4s/step - loss: 0.1309 - balanced_accuracy: 0.9564 - val_loss: 0.8725 - val_balanced_accuracy: 0.7461
Epoch 7/30
39/39 [============================

In [243]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

40/40 [==============================] - 28s 675ms/step


array([0.05940568, 0.08012749, 0.9989439 ], dtype=float32)

In [244]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.Trend

In [171]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted)) 

              precision    recall  f1-score   support

           0       0.91      0.70      0.79       466
           1       0.66      0.80      0.73       422
           2       0.80      0.83      0.81       392

    accuracy                           0.77      1280
   macro avg       0.79      0.78      0.78      1280
weighted avg       0.79      0.77      0.78      1280

